In [1]:
import getpass  # For input prompt not to show what is entered
import json     # Provide convinent functions to handle json objects 
import re       # For regular expression
import requests # To handle http requests
import unittest # For the test suite

import warnings
warnings.simplefilter('ignore') # Because I am using unverified ssl certificates 

baseurl = 'http://localhost:8080/api/v0.0.3-dev' # Pick yours

credentials = {
    'username': input('username: '), 
    'password': getpass.getpass('password: ')
}

username: admin
password: ········


# /login

In [2]:
class TestLogin(unittest.TestCase):

    def testSuccessful(self):
        payload = {}
        payload['username'] = credentials['username']
        payload['password'] = credentials['password']
        response = requests.post(
            url = baseurl + '/login',
            json = payload,
            verify = False
        )
        self.assertEqual(response.status_code, 200)
        data = json.loads(response.content.decode('ascii'))
        self.assertIn('token',data)

    def testWrongUsername(self):
        payload = {}
        payload['username'] = 'wrong-username'
        payload['password'] = credentials['password']
        response = requests.post(
            url = baseurl + '/login',
            json = payload,
            verify = False
        )
        self.assertEqual(response.status_code, 401)
        data = json.loads(response.content.decode('ascii'))
        self.assertNotIn('token',data)
        self.assertIn('error',data)
        self.assertEqual(data['error'], 'Incorrect username or password.')

def loginSuite():
    suite = unittest.TestSuite()
    suite.addTest(TestLogin('testSuccessful'))
    suite.addTest(TestLogin('testWrongUsername'))
    return suite

if __name__ == '__main__':
    runner = unittest.TextTestRunner()
    runner.run(loginSuite())


F.
FAIL: testSuccessful (__main__.TestLogin)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-2-d84096e0ad2a>", line 12, in testSuccessful
    self.assertEqual(response.status_code, 200)
AssertionError: 500 != 200

----------------------------------------------------------------------
Ran 2 tests in 0.531s

FAILED (failures=1)


# /projects

In [24]:
class TestProjects(unittest.TestCase):
    token = ''
    
    def setUp(self):
        TestProjects.token = json.loads(requests.post(
            url = baseurl + '/login',
            json = credentials,
            verify = False
        ).content.decode('ascii'))['token']

    def testProjects(self):
        response = requests.get(
            url = baseurl + '/projects',
            headers = {'Authorization': 'Bearer %s' % TestProjects.token},
            verify = False
        )
        self.assertEqual(response.status_code, 200)
        data = json.loads(response.content.decode('ascii'))
        self.assertIn('Projects',data)
        self.assertIn('loris',data['Projects'])
        self.assertIn('useEDC',data['Projects']['loris'])
        self.assertIn('PSCID',data['Projects']['loris'])

    def testProject(self):
        projectname = 'loris'
        response = requests.get(
            url = baseurl + '/projects/' + projectname,
            headers = {'Authorization': 'Bearer %s' % TestProjects.token},
            verify = False
        )
        self.assertEqual(response.status_code, 200)
        data = json.loads(response.content.decode('ascii'))
        self.assertIn('Meta',data)
        self.assertIn('Project',data['Meta'])
        self.assertEquals('loris',data['Meta']['Project'])
        self.assertIn('Visits',data)
        self.assertTrue(isinstance(data['Visits'],list))
        self.assertIn('Instruments',data)
        self.assertTrue(isinstance(data['Instruments'],list))
        self.assertIn('Candidates',data)
        self.assertTrue(isinstance(data['Candidates'],list))

    def testProjectCandidates(self):
        projectname = 'loris'
        response = requests.get(
            url = baseurl + '/projects/' + projectname + '/candidates',
            headers = {'Authorization': 'Bearer %s' % TestProjects.token},
            verify = False
        )
        self.assertEqual(response.status_code, 200)
        data = json.loads(response.content.decode('ascii'))
        self.assertIn('Candidates',data)
        self.assertTrue(isinstance(data['Candidates'],list))

    def testProjectImages(self):
        projectname = 'loris'
        response = requests.get(
            url = baseurl + '/projects/' + projectname + '/images',
            headers = {'Authorization': 'Bearer %s' % TestProjects.token},
            verify = False
        )
        self.assertEqual(response.status_code, 200)
        data = json.loads(response.content.decode('ascii'))
        self.assertIn('Images',data)

    def testProjectImagesSince(self):
        projectname = 'loris'
        response = requests.get(
            url = baseurl + '/projects/' + projectname + '/images?since=2011-10-10',
            headers = {'Authorization': 'Bearer %s' % TestProjects.token},
            verify = False
        )
        self.assertEqual(response.status_code, 200)
        data = json.loads(response.content.decode('ascii'))
        self.assertIn('Images',data)
        
    def testProjectInstruments(self):
        projectname = 'loris'
        response = requests.get(
            url = baseurl + '/projects/' + projectname + '/instruments',
            headers = {'Authorization': 'Bearer %s' % TestProjects.token},
            verify = False
        )
        self.assertEqual(response.status_code, 200)
        data = json.loads(response.content.decode('ascii'))
        self.assertIn('Instruments',data)

    def testProjectInstrument(self):
        projectname = 'loris'
        instrumentname = 'test_all_fields'
        response = requests.get(
            url = baseurl + '/projects/' + projectname + '/instruments/' + instrumentname,
            headers = {'Authorization': 'Bearer %s' % TestProjects.token},
            verify = False
        )
        self.assertEqual(response.status_code, 200)
        data = json.loads(response.content.decode('ascii'))
        self.assertIn('Meta',data)
        self.assertIn('Elements',data)
        
    def testProjectVisits(self):
        projectname = 'loris'
        response = requests.get(
            url = baseurl + '/projects/' + projectname + '/visits',
            headers = {'Authorization': 'Bearer %s' % TestProjects.token},
            verify = False
        )
        self.assertEqual(response.status_code, 200)
        data = json.loads(response.content.decode('ascii'))
        self.assertIn('Visits',data)
        
def projectsSuite():
    suite = unittest.TestSuite()
    suite.addTest(TestProjects('testProjects'))
    suite.addTest(TestProjects('testProject'))
    suite.addTest(TestProjects('testProjectCandidates'))
    suite.addTest(TestProjects('testProjectImages'))
    suite.addTest(TestProjects('testProjectImagesSince'))
    suite.addTest(TestProjects('testProjectInstruments'))
    suite.addTest(TestProjects('testProjectInstrument'))
    suite.addTest(TestProjects('testProjectVisits'))
    return suite


if __name__ == '__main__':
    runner = unittest.TextTestRunner()
    runner.run(projectsSuite())

.....F..
FAIL: testProjectInstruments (__main__.TestProjects)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-24-c77174f3a730>", line 84, in testProjectInstruments
    self.assertEqual(response.status_code, 200)
AssertionError: 500 != 200

----------------------------------------------------------------------
Ran 8 tests in 3.647s

FAILED (failures=1)


# /candidates

In [17]:
class TestCandidates(unittest.TestCase):
    token = ''
    
    def setUp(self):
        TestCandidates.token = json.loads(requests.post(
            url = baseurl + '/login',
            json = credentials,
            verify = False
        ).content.decode('ascii'))['token']

    def testCandidates(self):
        response = requests.get(
            url = baseurl + '/candidates',
            headers = {'Authorization': 'Bearer %s' % TestCandidates.token},
            verify = False
        )
        self.assertEqual(response.status_code, 200)
        data = json.loads(response.content.decode('ascii'))
        self.assertIn('Candidates',data)
        self.assertTrue(isinstance(data['Candidates'],list))

    def testPOSTCandidate(self):
        payload = {
            'Candidate' : {
                'Project' : 'loris',
                'DoB'     : "2015-09-10",
                'EDC'     : "2015-09-10",
                'Gender'  : "Female",
                'Site'    : 'Ottawa'
            }
        }

        response = requests.post(
            url = baseurl + '/candidates/',
            json = payload,
            headers = {'Authorization': 'Bearer %s' % TestCandidates.token},
            verify = False
        )
        self.assertEqual(response.status_code, 200)
        data = json.loads(response.content.decode('ascii'))
        self.assertIn('Meta',data)
        self.assertIn('CandID',data['Meta'])
        self.assertTrue(isinstance(data['Meta']['CandID'],int))
        
    def testCandidate(self):
        response = requests.get(
            url = baseurl + '/candidates/300001',
            headers = {'Authorization': 'Bearer %s' % TestCandidates.token},
            verify = False
        )
        self.assertEqual(response.status_code, 200)
        data = json.loads(response.content.decode('ascii'))
        self.assertIn('Meta',data)
        self.assertIn('Visits',data)
        self.assertTrue(isinstance(data['Visits'],list))

    def testCandidateVisit(self):
        response = requests.get(
            url = baseurl + '/candidates/300001/V01',
            headers = {'Authorization': 'Bearer %s' % TestCandidates.token},
            verify = False
        )
        self.assertEqual(response.status_code, 200)
        data = json.loads(response.content.decode('ascii'))
        self.assertIn('Meta',data)
        self.assertIn('CandID',data['Meta'])
        
def candidatesSuite():
    suite = unittest.TestSuite()
    suite.addTest(TestCandidates('testCandidates'))
    suite.addTest(TestCandidates('testCandidate'))
    suite.addTest(TestCandidates('testPOSTCandidate'))
    suite.addTest(TestCandidates('testCandidateVisit'))
    return suite

if __name__ == '__main__':
    runner = unittest.TextTestRunner()
    runner.run(candidatesSuite())

....
----------------------------------------------------------------------
Ran 4 tests in 2.020s

OK
